In [14]:
from experiment.modules.DataLoader import DataLoader
# import matplotlib.pyplot as plt
from datetime import date
import polars as pl
import altair as alt


data_loader = DataLoader()
metadata_df = data_loader.load_bodmas_metadata()

metadata_df = metadata_df.select(
        [pl.col("date"), pl.col("is_malware")]
        )

# malware_dates = metadata_df.filter(pl.col("is_malware")).select("date").to_series()
# goodware_dates = metadata_df.filter(pl.col("is_malware").not_()).select("date").to_series()

# plt.figure(figsize=(12, 6))
# plt.hist([malware_dates.to_numpy(), goodware_dates.to_numpy()], bins=30, label=['Malware', 'Goodware'])
# plt.xlabel("Date")
# plt.ylabel("Frequency")
# plt.legend()
# plt.xticks(rotation=45)
# plt.tight_layout()
# plt.savefig("timestamp_histogram.png")
# plt.show()

In [15]:
expr_before_2020 = pl.col("date").lt(date(2020, 1, 1))
expr_after_2020 = pl.col("date").gt(date(2020, 1, 1))

expr_first_half_2020 = pl.col("date").gt(date(2020, 1, 1)) & pl.col("date").lt(date(2020, 6, 1))
expr_second_half_2020_and_beyond = pl.col("date").gt(date(2020, 6, 1))
expr_is_malware = pl.col("is_malware")
expr_is_not_malware = pl.col("is_malware").not_()

expr_before_march = pl.col("date").lt(date(2020, 3, 1))
expr_after_march = pl.col("date").gt(date(2020, 3, 1))

In [16]:
t1_m = metadata_df.filter(expr_before_2020 & expr_is_malware).height
t1_g = metadata_df.filter(expr_before_2020 & expr_is_not_malware).height
t2_m = metadata_df.filter(expr_first_half_2020 & expr_is_malware).height
t2_g = metadata_df.filter(expr_first_half_2020 & expr_is_not_malware).height
t3_m = metadata_df.filter(expr_second_half_2020_and_beyond & expr_is_malware).height
t3_g = metadata_df.filter(expr_second_half_2020_and_beyond & expr_is_not_malware).height

tot = t1_m + t1_g + t2_m + t2_g + t3_m + t3_g

print(f"Before 2020:                        m = {t1_m}, g = {t1_g}, m_ratio = {round(t1_m / (t1_m + t1_g), 2)}, samples percentage = {round((t1_g + t1_m) / tot, 2)}")
print(f"First 6 months of 2020:             m = {t2_m}, g = {t2_g}, m_ratio = {round(t2_m / (t2_m + t2_g), 2)}, samples percentage = {round((t2_g + t2_m) / tot, 2)}")
print(f"Last 6 months of 2020 and later:    m = {t3_m}, g = {t3_g}, m_ratio = {round(t3_m / (t3_m + t3_g), 2)}, samples percentage = {round((t3_g + t3_m) / tot, 2)}")
print("\n")
print(f"All time: malwares = {t1_m + t2_m + t3_m}, goodwares = {t1_g + t2_g + t3_g}, total = {tot}")

Before 2020:                        m = 15823, g = 31328, m_ratio = 0.34, samples percentage = 0.37
First 6 months of 2020:             m = 23748, g = 18651, m_ratio = 0.56, samples percentage = 0.33
Last 6 months of 2020 and later:    m = 17454, g = 21103, m_ratio = 0.45, samples percentage = 0.3


All time: malwares = 57025, goodwares = 71082, total = 128107


In [17]:
plot_df = metadata_df.filter(expr_after_march)
plot_df_2 = metadata_df.filter(expr_after_2020 & ~expr_after_march)

In [ ]:
weekly_samples_after_february = plot_df.sort("date").group_by_dynamic(
    "date", 
    every="1w"
).agg([
    pl.col("is_malware").sum().alias("malware"),
    pl.col("is_malware").not_().sum().alias("goodware")
])

weekly_samples_first_2_months = plot_df_2.sort("date").group_by_dynamic(
    "date", 
    every="1w"
).agg([
    pl.col("is_malware").sum().alias("malware"),
    pl.col("is_malware").not_().sum().alias("goodware")
])

In [ ]:
base = alt.Chart(weekly_samples_after_february).encode(x='date:T')

malware_layer = base.mark_line(color='red').encode(y='malware:Q')
goodware_layer = base.mark_line(color='blue').encode(y='goodware:Q')

(malware_layer + goodware_layer).show()

alt.LayerChart(...)

In [23]:
base = alt.Chart(weekly_samples_first_2_months).encode(x='date:T')

malware_layer = base.mark_line(color='red').encode(y='malware:Q')
goodware_layer = base.mark_line(color='blue').encode(y='goodware:Q')

(malware_layer + goodware_layer).show()

alt.LayerChart(...)

In [21]:
from experiment.modules.DataLoader import DataLoader


data_loader = DataLoader()
metadata_df = data_loader.load_bodmas_metadata()
X, y = data_loader.load_bodmas_data()

X_train, y_train, test_sets = data_loader.split_data(metadata_df, X, y, train_cutoff=date(2020, 3, 1))

In [24]:
arrays = [X_train, y_train]

print("\nTest weeks:")

dates = [set_['period_start'].date() for set_ in test_sets]
print(dates[0])
print(dates[1])
print(dates[2])
print("...")
print(dates[-3])
print(dates[-2])
print(dates[-1])


Test weeks:
2020-03-01
2020-03-08
2020-03-15
...
2020-09-13
2020-09-20
2020-09-27
